## Timeseries analysis of StatsNZ trade data

In this notebook, I will examine StatsNZ trade data between January 2014 and September 2022. The trade data includes values of NZ Exports and Imports for goods and services per country and product codes. 

The table is available from the Stats NZ trade website: 
<a href="https://statisticsnz.shinyapps.io/trade_dashboard/">Stats NZ Trade Dashboard</a>


#### Importing data and exploring columns

In [1]:
#importing python libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
#reading data into dataframe df

file = '../data/Tradedata.csv'
df = pd.read_csv(file, parse_dates=['Period'], index_col='Period')   

In [3]:
df.head(5)

,Country Code,Country,Account,Category,Code,Flag,Total Value (NZD),Description
Period,,,,,,,,
201406,AF,Afghanistan,Imports,Goods,00,F,131905.0,Total
201406,AF,Afghanistan,Imports,Goods,08,F,107337.0,Fruits and nuts
201406,AF,Afghanistan,Imports,Goods,12,F,316.0,Oil seeds
201406,AF,Afghanistan,Imports,Goods,30,F,21786.0,Pharmaceuticals
201406,AF,Afghanistan,Imports,Goods,42,F,156.0,Leather products


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413815 entries, 201406 to 202209
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Country Code       413179 non-null  object 
 1   Country            413751 non-null  object 
 2   Account            413815 non-null  object 
 3   Category           413815 non-null  object 
 4   Code               413815 non-null  object 
 5   Flag               413815 non-null  object 
 6   Total Value (NZD)  413815 non-null  float64
 7   Description        413815 non-null  object 
dtypes: float64(1), object(7)
memory usage: 28.4+ MB


The trade table contains 413815 rows and 8 columns. The table does not have null values in the existing columns. 

In [5]:
#creating data frames for exported and imported goods

goods_exports= df[(df["Account"]=='Exports') & (df["Category"]=='Goods') ]
goods_imports= df[(df["Account"]=='Imports') & (df["Category"]=='Goods') ]

#size of dataframes
print('goods_exports:', goods_exports.shape, 'goods_imports:', goods_imports.shape)

goods_exports: (164196, 8) goods_imports: (185076, 8)


In [6]:
#Some statistics

print('Time Interval is between:', goods_exports.index.min(), 'quarter and ', goods_exports.index.max(), 'quarter')


#number of countries per quarter where goods are exported
countries_perquarter = goods_exports.groupby('Period')['Country'].nunique()
display(countries_perquarter)


Time Interval is between: 201406 quarter and  202209 quarter


Period
201406    217
201409    211
201412    210
201503    208
201506    205
201509    209
201512    207
201603    208
201606    214
201609    209
201612    209
201703    209
201706    211
201709    211
201712    207
201803    209
201806    211
201809    214
201812    218
201903    207
201906    210
201909    216
201912    210
202003    212
202006    206
202009    206
202012    212
202103    210
202106    206
202109    208
202112    210
202203    202
202206    206
202209    202
Name: Country, dtype: int64

In [7]:
#creating data frames for exported and imported services

services_exports= df[(df["Account"]=='Exports') & (df["Category"]=='Services') ]
services_imports= df[(df["Account"]=='Imports') & (df["Category"]=='Services') ]

#size of dataframes
print('services_exports:', services_exports.shape, 'services_imports:', services_imports.shape)

services_exports: (35756, 8) services_imports: (28787, 8)


In [8]:
#Some statistics

print('Time Interval is between:', services_exports.index.min(), 'quarter and ', services_exports.index.max(), 'quarter')


#number of countries per quarter where services are exported
countries_perquarter = services_exports.groupby('Period')['Country'].nunique()


Time Interval is between: 201406 quarter and  202209 quarter
